In [28]:
register_path = "/data/students/bigdata-01QYD/Lab6_DBD/register.csv"
stations_path = "/data/students/bigdata-01QYD/Lab6_DBD/stations.csv"
outputPath = "res_out_Lab7_SQL/"
treshold = 0.4

In [6]:
registerDF = spark.read.load(register_path,\
                            format="csv",\
                            sep="\t",\
                            header=True,\
                            inferSchema=True)\
.createOrReplaceTempView("Register")

In [7]:
registerFiltered = spark.sql("SELECT * \
    FROM Register \
    WHERE used_slots <> 0 OR free_slots <> 0")

In [44]:
def fullFunction(free_slots):
    if free_slots==0:
        return 1
    else:
        return 0
    
spark.udf.register("full", fullFunction)

<function __main__.fullFunction(free_slots)>

In [12]:
registerFiltered.createOrReplaceTempView("RegisterFiltered")

In [45]:
stationTS_DF = spark.sql("SELECT station, date_format(timestamp, 'EE') as dayofweek, hour(timestamp) as hour, full(free_slots) as fullstatus \
                          FROM Registerfiltered").createOrReplaceTempView("stationTSWithStatus") 

In [46]:
filteredWithTresholdDF = spark.sql("SELECT station, dayofweek, hour, avg(fullstatus) AS criticality \
        FROM stationTSwithstatus \
        GROUP BY station, dayofweek, hour \
        HAVING avg(fullstatus) > "+str(treshold))

##--------------------WE CAN COMPUTE JUST ONE QUERY--------------------
selectedPairsDF = spark.sql("""SELECT station, date_format(timestamp,'EE') as dayofweek, 
hour(timestamp) as hour, avg(full(free_slots)) as criticality 
FROM readings 
WHERE free_slots<>0 OR used_slots<>0
GROUP BY station, date_format(timestamp,'EE'), hour(timestamp)
HAVING avg(full(free_slots))>"""+str(threshold))

#---------------------------------------------------------------------------

In [47]:
filteredWithTresholdDF.createOrReplaceTempView("FilteredRegister")

In [48]:
stationsDF = spark.read.load(stations_path,\
                             format="csv",\
                             sep="\t",\
                             header=True,\
                             inferSchema=True
                            )\
.createOrReplaceTempView("Stations")

In [52]:
outputDF = spark.sql("SELECT station, dayofweek, hour, criticality, longitude, latitude \
        FROM filteredregister INNER JOIN Stations ON \
        filteredregister.station = stations.id \
        ORDER BY criticality desc, station, dayofweek, hour")

In [53]:
outputDF.coalesce(1).write.csv(outputPath, header=True)